In [1]:
import pandas as pd

from pendulum import now

In [2]:
dataset_name = 'sidra_tab5457_sementes'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

Área de colheita/plantada (ha)

### Bronze

In [3]:
contador = 1

ultimo_ano = now().year - 1
for ano in range(2000, ultimo_ano):
    if contador == 1:
        df_5457 = pd.read_json(f'https://apisidra.ibge.gov.br/values/t/5457/n6/all/v/216/p/{ano}/c782/40102,40106,40112,40119,40122,40124,40127')
        contador += 1
    else:
        df_5457 = pd.concat(
            [df_5457,
            (pd.read_json(f'https://apisidra.ibge.gov.br/values/t/5457/n6/all/v/216/p/{ano}/c782/40102,40106,40112,40119,40122,40124,40127')
             .iloc[1:])],
            ignore_index=True            
        )
        contador += 1

df_5457.to_json(f'{raiz}/bronze_{dataset_name}.json')
print(f'Foram contabilizadas {contador} repetições.')

Foram contabilizadas 24 repetições.


### Prata

In [4]:
(pd.read_json(f'{raiz}/bronze_{dataset_name}.json')
 .drop(0, axis = 0)
 .filter(['D1C', 'D3C', 'D4C', 'V'])
 .assign(**{
        'uf': lambda df: df['D1C'].str.slice(0,2).astype(int),
        'hectare': lambda df: pd.to_numeric(df['V'], errors='coerce'),
        'municipio': lambda df: df['D1C'].astype(int),
        'ano': lambda df: df['D3C'].astype(int),
        'cultura': lambda df: df['D4C'].astype(int),
        'fonte': 'Sementes'
        })
 .dropna(subset='hectare')
 .convert_dtypes()
 .groupby(['uf', 'municipio', 'ano', 'cultura', 'fonte'])
 .agg({'hectare':sum})
 .to_parquet(f'{raiz}/prata_{dataset_name}.parquet')
 )

### Ouro

Quantidade de sementes/mudas utilizadas por área para a plantação das principais culturas agrícolas do país.

((Área * Quantidade_Sementes_ha)/1000) * (Teor N,P,K/1000)

In [3]:
parametro = {
    40102: {'semente_ha': 160, 'n': 13, 'p': 2.9, 'k': 3}, # Arroz
    40106: {'semente_ha': 12500, 'n': 1, 'p': 0.3, 'k': 1.5}, # Cana-de-açúcar
    40112: {'semente_ha': 45, 'n': 50, 'p': 5.7, 'k': 12.5}, # Feijão
    40119: {'semente_ha': 750, 'n': 2.1, 'p': 0.2, 'k': 2}, # Mandioca
    40122: {'semente_ha': 23, 'n': 12, 'p': 2.7, 'k': 3.7}, # Milho
    40124: {'semente_ha': 70, 'n': 55, 'p': 5.2, 'k': 16.6}, # Soja
    40127: {'semente_ha': 120, 'n': 21, 'p': 4.1, 'k': 4.6}, # Trigo
    }

In [4]:
df = (pd.read_parquet(f'{raiz}/prata_{dataset_name}.parquet')
      .groupby(['uf', 'ano', 'cultura', 'fonte'])
      .agg({'hectare':sum})
      .reset_index()
      .assign(**{
          'N': lambda df: df.apply(lambda lin: ((lin['hectare']*parametro[lin['cultura']]['semente_ha'])/1000) * (parametro[lin['cultura']]['n']/1000), axis = 1),
          'P': lambda df: df.apply(lambda lin: ((lin['hectare']*parametro[lin['cultura']]['semente_ha'])/1000) * (parametro[lin['cultura']]['p']/1000), axis = 1),
          'K': lambda df: df.apply(lambda lin: ((lin['hectare']*parametro[lin['cultura']]['semente_ha'])/1000) * (parametro[lin['cultura']]['k']/1000), axis = 1)
          })
      .drop('hectare', axis = 1)
      .melt(id_vars=['uf', 'ano', 'cultura', 'fonte'],
            value_vars=['N', 'P', 'K'],
            var_name='elemento',
            value_name='valor')
      .convert_dtypes()
      .groupby(['uf', 'ano', 'fonte', 'elemento'])
      .agg({'valor': sum})
      .reset_index(level='elemento')
      .sort_values(by=['elemento', 'uf'],
                   key=lambda x: x.map({'N': 0, 'P': 1, 'K': 2}))
      )

df.to_csv(f'{raiz}/ouro_npk_sementes_ha.csv')

df

elemento       valor
uf ano  fonte                        
11 2000 Sementes        N  506.300993
   2001 Sementes        N  443.515866
   2002 Sementes        N  403.313943
   2003 Sementes        N  487.421785
   2004 Sementes        N  610.753246
...                   ...         ...
53 2018 Sementes        K   103.98166
   2019 Sementes        K  106.557425
   2020 Sementes        K   109.05712
   2021 Sementes        K   110.84075
   2022 Sementes        K     119.393

[1863 rows x 2 columns]